In [1]:
from pandas import read_csv
import pandas as pd
from sklearn.model_selection import train_test_split
import autogluon as ag
from autogluon import TabularPrediction as task

C:\Users\oabez\Anaconda3\lib\site-packages\mxnet\optimizer\optimizer.py:167: UserWarning: WARNING: New optimizer gluonnlp.optimizer.lamb.LAMB is overriding existing optimizer mxnet.optimizer.optimizer.LAMB
  Optimizer.opt_registry[name].__name__))


In [2]:
test = read_csv("NTI_data/test.csv")
train_target = read_csv("NTI_data/train_target.csv")
transactions_test = read_csv("NTI_data/transactions_test.csv")
transactions_train = read_csv("NTI_data/transactions_train.csv")

In [3]:
#Train
#Признаки, связанные с суммой покупки
agg_features=transactions_train.groupby('client_id')['amount_rur'].agg(['sum','mean','max']).reset_index()

#Количество в каждой категории
amount_of_purchase=transactions_train.groupby(['client_id','small_group'])['amount_rur'].count()
counts_train=amount_of_purchase.reset_index().pivot(index='client_id', \
                                                      columns='small_group',values='amount_rur')
counts_train=counts_train.fillna(0)
counts_train.columns=['small_group_' + str(i) for i in counts_train.columns]

#Сумма в каждой категории
sum_of_group=transactions_train.groupby(['client_id','small_group'])['amount_rur'].sum()
sum_train=sum_of_group.reset_index().pivot(index='client_id', \
                                                      columns='small_group',values='amount_rur')
sum_train=sum_train.fillna(0)
sum_train.columns=['small_group_sum_'+str(i) for i in sum_train.columns]

#Минимум в каждой категории
min_of_group=transactions_train.groupby(['client_id','small_group'])['amount_rur'].min()
min_train=sum_of_group.reset_index().pivot(index='client_id', \
                                                      columns='small_group',values='amount_rur')
min_train=min_train.fillna(0)
min_train.columns=['small_group_min_'+str(i) for i in min_train.columns]

#Максимум в каждой категории
max_of_group=transactions_train.groupby(['client_id','small_group'])['amount_rur'].min()
max_train=sum_of_group.reset_index().pivot(index='client_id', \
                                                      columns='small_group',values='amount_rur')
max_train=max_train.fillna(0)
max_train.columns=['small_group_max_'+str(i) for i in max_train.columns]

#Средняя сумма в каждой категории
avg_of_group=transactions_train.groupby(['client_id','small_group'])['amount_rur'].mean()
avg_train=avg_of_group.reset_index().pivot(index='client_id', \
                                                      columns='small_group',values='amount_rur')
avg_train=avg_train.fillna(0)
avg_train.columns=['small_group_avg_'+str(i) for i in avg_train.columns]

#Соединим все в одну таблицу
counts_train=pd.merge(sum_train,counts_train,on='client_id')
counts_train=pd.merge(avg_train,counts_train,on='client_id')
counts_train=pd.merge(min_train,counts_train,on='client_id')
counts_train=pd.merge(max_train,counts_train,on='client_id')
train=pd.merge(train_target,agg_features,on='client_id')
train=pd.merge(train,counts_train.reset_index(),on='client_id')

In [4]:
train_data = train.head(25000)
test_data = train.tail(5000)

In [ ]:
predictor = task.fit(train_data=train_data, label='bins', problem_type="multiclass", auto_stack=True)
performance = predictor.evaluate(test_data)

No output_directory specified. Models will be saved in: AutogluonModels/ag-20200309_164438/
Beginning AutoGluon training ...
AutoGluon will save models to AutogluonModels/ag-20200309_164438/
Train Data Rows:    25000
Train Data Columns: 1015
Preprocessing data ...
Feature Generator processed 25000 data points with 1009 features
Original Features:
	int features: 1
	float features: 1008
Generated Features:
	int features: 0
All Features:
	int features: 1
	float features: 1008
	Data preprocessing and feature engineering runtime = 2.92s ...
AutoGluon will gauge predictive performance using evaluation metric: accuracy
To change this, specify the eval_metric argument of fit()
AutoGluon will early stop models using evaluation metric: accuracy
C:\Users\oabez\Anaconda3\lib\imp.py:342: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  return _load(spec)
Fitting model: RandomForestClassifierGini_STA